<a href="https://colab.research.google.com/github/NikolayWTF/Hacato-SPB-Top-Blog/blob/main/%D0%A5%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD_%D1%81%D0%BF%D0%B1_%D0%B8%D1%82%D0%BE%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Загружаю данные для теста программы
!gdown 1R_tRU1JdZVyubJeiCaBRXce7_CPuHA33
!unrar x -Y "images_zn.rar" "images"

In [32]:
#Загружаю архив с весами всех моделей
!gdown 1RVWFEC2UoPSt7O5EwiKK_Wb527CWxCVn

Downloading...
From: https://drive.google.com/uc?id=1RVWFEC2UoPSt7O5EwiKK_Wb527CWxCVn
To: /content/networks.rar
100% 213M/213M [00:07<00:00, 27.3MB/s]


In [ ]:
#Устанавливаю все необходимые библиотеки
!pip install unrar
!pip install ultralytics
!pip install easyocr
!pip install pandas

In [33]:
#Распаковываю архив с моделями и добавляю в папку networks
!unrar x -Y "networks.rar" "networks"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from networks.rar

Creating    networks                                                  OK
Extracting  networks/yolov8m-tg_v2-30e.pt                                  1%  3%  5%  7%  9% 11% 13% 15% 17% 19% 21% 22%  OK 
Extracting  networks/yolov8m-vk-30e.pt                                    24% 26% 28% 30% 32% 34% 36% 38% 40% 42% 44% 45%  OK 
Extracting  networks/yolov8m-yt1_v5-30e.pt                                47% 49% 51% 53% 55% 57% 58% 60% 62% 64% 66% 67%  OK 
Extracting  networks/yolov8m-yt2_v3-30e.pt                                69% 71% 73% 75% 77% 79% 81% 83% 85% 87% 89% 90%  OK 
Extracting  networks/yolov8s-zn-gpu-15e.pt                                92% 94% 96%

In [27]:
# Импортирую необходимые библиотеки
import ultralytics
from ultralytics import YOLO
import numpy as np
import cv2
import os
import easyocr
import pandas as pd

In [ ]:
def folder_handling(mode):
  #Работа с папкой
  column_folder = []
  column_folder_zn = []
  for filename in photos:
    file_extension = filename.split(".")[1].lower()

    if file_extension not in ban_extension:
      path_to_img = directory_name + filename

      results = model(path_to_img)

      for r in results:
        boxes = r.boxes
        if len(boxes) == 0:
          column_folder.append(filename)
          column_folder_zn.append("Данные для аналитики не найдены")
          # print(filename, "Данные для аналитики не найдены")
        else:
          for box in boxes:
            x_0 = int(box.xyxy[0][0])
            y_0 = int(box.xyxy[0][1])
            x_1 = int(box.xyxy[0][2])
            y_1 = int(box.xyxy[0][3])
            img = r.orig_img[y_0:y_1, x_0:x_1]

            #Возможно сюда надо не изображение, а имя файла с изображением
            cv2.imwrite("tmp.jpg", img)
            ans = reader.readtext("tmp.jpg")

            if len(ans) == 0:
              filename_def, zn_def = contrast(filename)
              column_folder.append(filename_def)
              column_folder_zn.append(zn_def)
            else:
              for a in ans:
                # print(filename, a[1])
                column_folder.append(filename)
                column_folder_zn.append(a[1])
  find_parametr = ""
  final_filename = ""
  if mode == "zn":
    find_parametr = "Дочитывания"
    final_filename = "./zn.xlsx"
  elif mode == "vk":
    find_parametr = "Подписчики"
    final_filename = "./vk.xlsx"
  elif mode == "tg":
    find_parametr = "ERR/VR"
    final_filename = "./tg.xlsx"
  elif mode == "yt1":
    find_parametr = "Подписчики"
    find_parametr = "Просмотры"
    final_filename = "./yt1.xlsx"
  elif mode == "yt2":
    find_parametr = "Просмотры"
    final_filename = "./yt2.xlsx"

  df = pd.DataFrame({"Имя файла" : column_folder,
                    find_parametr : column_folder_zn})
  df.to_excel(final_filename, index=False)


def contrast(filename):
  # Это контрастность и яркость, применяется, если ocr ничего не распознал
  image_bgr = cv2.imread("tmp.jpg", cv2.IMREAD_COLOR)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  contrast_matrix = np.ones(image_rgb.shape) * 1.9
  brightness_matrix = np.ones(image_rgb.shape, dtype="uint8") * 140
  image_rgb_br = cv2.subtract(image_rgb, brightness_matrix)
  image_rgb_con = np.uint8(np.clip(cv2.multiply(np.float64(image_rgb_br), contrast_matrix), 0, 255))
  ans = reader.readtext(image_rgb_con)

  if len(ans) == 0:
    ans = reader.readtext(image_rgb_br)
    if len(ans) == 0:
      return (filename, "Данные не распознаны")
    else:
      for a in ans:
        return (filename, a[1])
  else:
    for a in ans:
      return (filename, a[1])

mode = "yt" # Тут выбираем с какой соц сетью работаем. Режимы: zn, vk, tg, yt
content = "folder" # С чем работаем с 1 фото или с папкой. Режимы: photo, folder

reader = easyocr.Reader(['ru', 'en'], gpu = True) #Подключаю EasyOCR
directory_name = 'images/' # Имя папки с файлами
photos = os.listdir(directory_name)
ban_extension = ["doc", "docx", "pdf", "pptx", "heic", "zip", "jfif"]
if mode == "zn":

  model_name = 'networks/yolov8s-zn-gpu-15e.pt' # Выбрал модель для Дзена
  model = YOLO(model_name)

  if content == "folder":

    folder_handling(mode)

elif mode == "vk":

  model_name = 'networks/yolov8m-vk-30e.pt' # Выбрал модель для ВК
  model = YOLO(model_name)

  if content == "folder":

    folder_handling(mode)

elif mode == "tg":

  model_name = 'networks/yolov8m-tg_v2-30e.pt' # Выбрал модель для Телеграма
  model = YOLO(model_name)

  if content == "folder":

    folder_handling(mode)

elif mode == "yt":

  model_name = 'networks/yolov8m-yt1_v5-30e.pt' # Выбрал модель для Ютуба
  model = YOLO(model_name)

  if content == "folder":

    folder_handling("yt1")

    model_name = 'networks/yolov8m-yt2_v3-30e.pt' # Выбрал модель для Ютуба
    model = YOLO(model_name)

    folder_handling("yt2")

  # ВАЖНО. ПОСЛЕ ПРОГОНА 1й МОДЕЛИ, СЮДА НУЖНО ВСТАВИТЬ yt2, но пока нет весов

else:
  print("ОШИБКА выбрана не существующая сеть")